### Project Chicago: Transform ResStock PACKAGE Results to Multi-family (RECS) Building Types
Created on: 01/07/2020 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/ResStock/files


### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/chicagoeui
* processed result tables for plots: https://nrel.sharepoint.com/sites/ChicagoRetrofits/Shared%20Documents/Forms/AllItems.aspx?viewid=289cdd1a%2D97c9%2D4bcc%2D8416%2Dc19bf01c6302&id=%2Fsites%2FChicagoRetrofits%2FShared%20Documents%2FGeneral%2FUpgrade%20results

### Initialize
For modifying plots, can go directly to section 2.3 after initialization

In [2]:
# local path to downloaded results
iteration = '03_mf' # <----- options: '01'
iter_path = f'cookcnty_packages_{iteration}'
result_dir = '/Users/lliu2/Documents/Chicago retrofits/ResStock results'
result_path = os.path.join(result_dir, iter_path)

## create folder for post-processed results:
if not os.path.exists(os.path.join(result_path, 'processed results')):
    os.mkdir(os.path.join(result_path, 'processed results'))
    
## create folder for exported baseline and upgrade results to csv (to share with Elevate):
combined_res_csv_path = os.path.join(result_path, 'processed results', 'raw combined csvs')
if not os.path.exists(combined_res_csv_path):
    os.mkdir(combined_res_csv_path)
    
## create folder for plots:
plot_path = os.path.join(result_path, 'processed results', 'plots')
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

print(f'Results path: \n   {result_path}')


Results path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_mf


### Functions

In [3]:
def get_per_unit_sim_output(df, ref):
    """
    ref (df): baseline df
    """
    cols = [x for x in df.columns if 
            x.endswith('_kwh') or
            x.endswith('_mbtu') or
            x.endswith('_therm') or
            x.endswith('_cost_usd') or
            x.endswith('_ft_2')
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output"')

def get_per_unit_sim_output_limited(df, ref):
    """
    To reduce computing time
    ref (df): baseline df
    """
    cols = ['simulation_output_report.total_site_natural_gas_therm',
            'simulation_output_report.total_site_electricity_kwh',
            'simulation_output_report.total_site_energy_mbtu',
            'simulation_output_report.upgrade_cost_usd',
            'simulation_output_report.floor_area_conditioned_ft_2'
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output_limited"')

def add_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    df['gas_eui_thermpersqft'] = df['simulation_output_report.total_site_natural_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['simulation_output_report.total_site_electricity_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_kbtupersqft'] = df['simulation_output_report.total_site_energy_mbtu'].divide(df['sqft'])*1000 # kbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_sqft_eui"')

def get_res_by_prototype(df, filter_by_df, row):
    """
    ARG:
        df (dataframe): df to slice on
        filter_by_df (dataframe): df used to do the slicing
    RETURN:
        filtered df (dataframe)
    """
    slice_by_df = filter_by_df.copy().set_index('building_id').reindex(df['building_id']).reset_index()
    res_group_i = df[slice_by_df['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        slice_by_df['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        slice_by_df['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        slice_by_df['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    res_group_i = res_group_i[res_group_i['completed_status']=="Success"]
    
    return res_group_i
print('func loaded: "get_res_by_prototype"')

def load_upgrade(n, file_dir=result_path):
    """
    ARG:
        n (int, str): upgrade number
        file_dir (str): folder in which upgrade can be found, default to main result dir
    RETURN:
        df (dataframe) of upgrade n
    """
    df = pd.read_parquet(os.path.join(file_dir,'upgrades',
                                     f'upgrade={n}/results_up{n:02d}.parquet'))
    return df
print('func loaded: "load_upgrade"')



func loaded: "get_per_unit_sim_output"
func loaded: "get_per_unit_sim_output_limited"
func loaded: "add_sqft_eui"
func loaded: "get_res_by_prototype"
func loaded: "load_upgrade"


### 1. BASELINE results

In [4]:
save_a_copy_in_csv = True # <-----
res = pd.read_parquet(os.path.join(result_path,'baseline','results_up00.parquet'))

# (1) get sqft, gas/elec/site eui
res = add_sqft_eui(res, res)
res['build_existing_model.sample_weight'] = 2173432/40000

if save_a_copy_in_csv:
    res.to_csv(os.path.join(combined_res_csv_path,'results_baseline.csv'), index=False)
    
# (1) get sim output at the unit level (req for MF)
res = get_per_unit_sim_output(res, res)

# (2) get sqft, gas/elec/site eui - redo to get unit-level results
res = add_sqft_eui(res, res)

# check
Nbldgs = 20759
jobs_missing = set(range(1,100)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {Nbldgs-len(res)} buildings ({((Nbldgs-len(res))/Nbldgs*100):.2f}%)')

res

- 0 jobs missing: set()
- 0 buildings (0.00%)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft
0,1,54,2021-05-23 13:56:59,2021-05-23 13:58:28,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.636364,17.909091,35.439281,27.020360,4793.0,39.621348,1138.0,1.361860,9.549645,168.738275
1,2,34,2021-05-23 19:33:59,2021-05-23 19:49:34,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.272727,17.272727,157.101383,111.074527,5128.0,191.819981,1138.0,0.452656,4.967888,62.205923
2,3,5,2021-05-23 17:18:14,2021-05-23 17:21:42,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.181818,18.181818,102.139434,84.674224,6471.0,108.604937,2115.0,0.589481,5.330745,77.123277
3,4,25,2021-05-23 21:02:19,2021-05-23 21:03:43,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.454545,17.818182,22.218383,13.834917,4932.0,24.872681,853.0,0.771258,7.795361,103.706277
4,5,35,2021-05-23 13:04:54,2021-05-23 13:26:08,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.272727,10.818182,259.820679,686.335258,619.0,880.114657,853.0,0.000000,16.979473,57.936321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,34,2021-05-24 02:51:17,2021-05-24 02:58:56,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.090909,10.454545,215.333446,308.773113,873.0,407.545486,1138.0,0.018703,21.477401,75.153764
20755,20756,95,2021-05-23 12:23:58,2021-05-23 13:09:50,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,12.909091,13.636364,670.480014,990.287448,641.0,1103.620275,617.0,0.031215,20.473324,72.978604
20756,20757,44,2021-05-23 09:49:28,2021-05-23 10:04:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.545455,13.545455,757.613102,672.694845,5103.0,886.638635,1138.0,0.424486,5.293215,60.499668
20757,20758,50,2021-05-23 12:15:28,2021-05-23 12:26:20,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.636364,6.000000,167.620016,778.661058,630.0,893.685975,853.0,0.000000,19.625330,66.964351


In [5]:
# assign heating/cooling
cooling_col = 'build_existing_model.hvac_cooling_type'
heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')

# if using a national run
if iteration == '_national_2018':
    res = res[res['build_existing_model.ahs_region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res
    

selected


### 1.1. Check housing charateristics distributions in BASELINE

In [6]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iter_path} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, 'baseline', 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, "baseline")}')

>>> ResStock - cookcnty_packages_03_mf - BASELINE result summary:

  * 20759 / 20759 samples ran successfully, 0 failed, efficacy: 100.0% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1           0.414374
2           0.200491
3           0.385134
N_failed    0.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry     0.877547
WoodStud    0.122453
N_failed    0.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59     0.147888
1960-79     0.248326
1980-99     0.141963
2000-09     0.091141
2010s       0.015560
<1940       0.355123
N_failed    0.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC    0.422756
Heat Pump     0.041476
None          0.088588
Room AC       0.447180
N_failed      0.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP                    

### 1.2. Add additional metrics to Baseline

In [7]:
## EE prototype tags
res['vintage_ee'] = '3: post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '2: 1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = '1: pre-1942'

res['stories'] = '2: 2+ stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = '1: <2 stories'

print('new cols added to "res"')
res

new cols added to "res"


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,54,2021-05-23 13:56:59,2021-05-23 13:58:28,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,35.439281,27.020360,4793.0,39.621348,1138.0,1.361860,9.549645,168.738275,2: 1942-1978,1: <2 stories
1,2,34,2021-05-23 19:33:59,2021-05-23 19:49:34,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,157.101383,111.074527,5128.0,191.819981,1138.0,0.452656,4.967888,62.205923,2: 1942-1978,2: 2+ stories
2,3,5,2021-05-23 17:18:14,2021-05-23 17:21:42,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,102.139434,84.674224,6471.0,108.604937,2115.0,0.589481,5.330745,77.123277,1: pre-1942,1: <2 stories
3,4,25,2021-05-23 21:02:19,2021-05-23 21:03:43,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,22.218383,13.834917,4932.0,24.872681,853.0,0.771258,7.795361,103.706277,2: 1942-1978,1: <2 stories
4,5,35,2021-05-23 13:04:54,2021-05-23 13:26:08,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,259.820679,686.335258,619.0,880.114657,853.0,0.000000,16.979473,57.936321,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,34,2021-05-24 02:51:17,2021-05-24 02:58:56,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,215.333446,308.773113,873.0,407.545486,1138.0,0.018703,21.477401,75.153764,2: 1942-1978,2: 2+ stories
20755,20756,95,2021-05-23 12:23:58,2021-05-23 13:09:50,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,670.480014,990.287448,641.0,1103.620275,617.0,0.031215,20.473324,72.978604,3: post-1978,2: 2+ stories
20756,20757,44,2021-05-23 09:49:28,2021-05-23 10:04:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,757.613102,672.694845,5103.0,886.638635,1138.0,0.424486,5.293215,60.499668,3: post-1978,2: 2+ stories
20757,20758,50,2021-05-23 12:15:28,2021-05-23 12:26:20,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,167.620016,778.661058,630.0,893.685975,853.0,0.000000,19.625330,66.964351,3: post-1978,2: 2+ stories


### Honnie's baseline bldgs of interest (IQR @ 50%)
* Brick - Pre-1942 1-2+ Stories
* Frame - Pre-1942 1-2+ Stories
* Brick Mid Century
* Brick 2-4 Flat Pre-1942 (here)
* Frame 2-4 Flat Pre-1942 (here)

In [8]:
metric = 'site_eui_kbtupersqft'
gb = ['build_existing_model.geometry_building_type_recs',
      'build_existing_model.geometry_wall_type',
      'vintage_ee']

query = res.query('completed_status=="Success"').groupby(gb)[metric].describe()
query.columns = [".".join([metric, x]) for x in query.columns]

query.to_csv(
    os.path.join(result_path, "baseline_results_by_bldgtype_walltype_vintage_mf.csv")
)

query

site_eui_kbtupersqft.count  \
build_existing_model.geometry_building_type_recs build_existing_model.geometry_wall_type vintage_ee                                 
Multi-Family with 2 - 4 Units                    Masonry                                 1: pre-1942                       2614.0   
                                                                                         2: 1942-1978                      2263.0   
                                                                                         3: post-1978                      1014.0   
                                                 WoodStud                                1: pre-1942                       1470.0   
                                                                                         2: 1942-1978                       227.0   
                                                                                         3: post-1978                       124.0   
Multi-Family with 5+ Units                       Masonry                                 1: pre-1942                       2959.0   
                                                                                         2: 1942-1978                      5516.0   
                                                                                         3: post-1978                      3851.0   
                                                 WoodStud                                1: pre-1942                        329.0   
                                                                                         2: 1942-1978                       219.0   
                                                                                         3: post-1978                       173.0   

                                                                                                       site_eui_kbtupersqft.mean  \
build_existing_model.geometry_building_type_recs build_existing_model.geometry_wall_type vintage_ee                                
Multi-Family with 2 - 4 Units                    Masonry                                 1: pre-1942                  121.615448   
                                                                                         2: 1942-1978                 109.522857   
                                                                                         3: post-1978                  75.255849   
                                                 WoodStud                                1: pre-1942                  130.221855   
                                                                                         2: 1942-1978                 112.757510   
                                                                                         3: post-1978                  73.009636   
Multi-Family with 5+ Units                       Masonry                                 1: pre-1942                  117.441136   
                                                                                         2: 1942-1978                  86.655314   
                                                                                         3: post-1978                  63.106811   
                                                 WoodStud                                1: pre-1942                  124.637522   
                                                                                         2: 1942-1978                  96.125553   
                                                                                         3: post-1978                  58.792206   

                                                                                                       site_eui_kbtupersqft.std  \
build_existing_model.geometry_building_type_recs build_existing_model.geometry_wall_type vintage_ee                               
Multi-Family with 2 - 4 Units                    Masonry                                 1: pre-1942                  39.951911  

## 2A. PACKAGE post-processing ###
Three packages were ran as partitioned files. See "Cookcnty_packages_02.yml" for info
#### (1) Packages WITHOUT partitions (N=1, pkg 06)

In [ ]:
def check_and_correct_for_missing_buildings(Pi, res, upgrade_no):
    delta = len(res)-len(Pi)
    if delta>0:
        print(f'   - upgrade_{upgrade_no:02d} is missing {delta} buildings compared to baseline')
    # always reindex
    Pi = Pi.set_index('building_id').reindex(res['building_id']).reset_index()
        
    return Pi

'func "check_and_correct_for_missing_buildings" loaded'

#### (2) Packages WITH single-level partitions (N=10, pkg 07-16)

In [ ]:
# 2 partitiions over 'Attic Insulation'

def combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades,
                                           MSHP_option=None, nullify_total_ng=False, save_as_csv=True, 
                                           recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        partition_para (str): para to filter the upgrades by and combine
        options_list_for_first_upgrades (list of list): list of para options to filter the first n-1 upgrades by
        MSHP_option (int): option number in package to correct, default to None
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N = len(upgrade_list)
    partition_para = f'build_existing_model.{partition_para}'
    
    # check setting:
    if N-1 != len(options_list_for_first_upgrades):
        raise ValueError(f'The size of "options_list_for_first_upgrades" = {len(options_list_for_first_upgrades)}, '+
                         f'is not 1 less than the size of "upgrade_list" = {N}')

    if not os.path.exists(filename) or recreate_from_scratch:
        print(f'building from resstock results... partitioned by:\n  "{partition_para}"')

        ## (1) building ids for each partitions based on partition_para and para_options
        bldg_list = options_list = []; msg = ' '; n_bldgs = 0
        for n in range(N-1):
            bldgs_for_part_n = res[res[partition_para].isin(options_list_for_first_upgrades[n])]['building_id']
            bldg_list.append(bldgs_for_part_n)
            
            options_for_part_n = options_list_for_first_upgrades[n]
            options_list = options_list + options_for_part_n
            
            n_bldgs += len(bldgs_for_part_n)
            msg += f' part {n+1}: {len(bldgs_for_part_n)},'
        
        # for last partition:
        bldgs_for_part_n = res[~res[partition_para].isin(options_list)]['building_id']
        bldg_list.append(bldgs_for_part_n)
        
        n_bldgs += len(bldgs_for_part_n)
        msg += f' part {N}: {len(bldgs_for_part_n)}, total: {n_bldgs}'
        print(msg)

        ## (2) combine partitions and update upgrade name
        P1 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P1.append(p[p['building_id'].isin(Bi)])

        P1 = pd.concat(P1, axis=0).sort_index()

        P1['apply_upgrade.upgrade_part'] = P1['apply_upgrade.upgrade_name'] # new col to show partition #
        P1['apply_upgrade.upgrade_name'] = P1['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P1.columns:
                P1['simulation_output_report.upgrade_cost_usd_orig'] = P1['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P1['simulation_output_report.upgrade_cost_usd'] = P1['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P1['simulation_output_report.upgrade_cost_usd'] -= P1[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P1['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P1.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P1.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P1['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P1.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P1['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, MSHP_cost]

            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P1.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
        
        ## (4) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P1.loc[P1['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0
        
        ## (5) save
        if save_as_csv:
            print(f'\n>> combined file saved to: {combined_res_csv_path}')
            P1.to_csv(filename, index=False)
            
    else:
        P1 = pd.read_csv(filename)

#     display(P1)
    return P1

print('func "combine_df_from_two_partitions"')


In [ ]:
pkg_no = 7
upgrade_list = [2, 3]
partition_para = 'insulation_unfinished_attic'
options_list_for_first_upgrades = [
    ['Uninsulated, Vented','Ceiling R-7, Vented','Ceiling R-13, Vented'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


In [ ]:
pkg_no = 11
upgrade_list = [1, 2]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

In [ ]:
pkg_no = 13
upgrade_list = [3, 4]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)

#### (3) Packages WITH multi-level partitions (N=4, pkg 17-20)

In [ ]:
def combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option=None, 
                                         fix_upgrade_name=False, nullify_total_ng=False, save_as_csv=True, 
                                         recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        level1_list (list of dict): list of dictionaries defining the level 1 partition key and options
        level2_list (list of dict): list of dictionaries defining the level 2 partition key and options
        MSHP_option (int): option number in package to correct, default to None
        fix_upgrade_name (bool): default to True, for pkg 21-24 only
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N_upgrades = len(upgrade_list)
    N_partitions = len(level1_list)*len(level2_list)
    if N_upgrades != N_partitions:
        raise ValueError(f'The size of "upgrade_list" = {N_upgrades} does not match the number of enumeration from '+
                        f'"level1_list" and "level2_list" = {N_partitions}')
    
    if not os.path.exists(filename) or recreate_from_scratch:
        
        print(f'building from resstock run results, {N_partitions} partitions...\n')

        ## building ids for each partitions
        bldg_list = []; CBi_len = 0

        ##### level 1
        P4C1 = level1_list # <----
        for i , Ci in enumerate(P4C1,1):
            CBi = set()
            for key, lst in Ci.items():
                Bi = res[res[f'build_existing_model.{key}'].replace({np.nan:'None'}).isin(lst)]['building_id']
                CBi = CBi.union(set(list(Bi)))
                print(f'- 1.{i} {key}, {len(Bi)} / {len(res)}')
            print(f'- 1.{i} total, {len(CBi)}')
            res1 = res[res[f'building_id'].isin(CBi)]

            ##### level 2
            P4C2 = level2_list # <----
            CBi_2all = set(); CBi_2len = 0
            for k, Ci in enumerate(P4C2,1):
                if k == len(P4C2):
                    CBi = set(list(res1['building_id'])).difference(CBi_2all)
                    print(f'    + 3.{k} total, {len(CBi)}')
                else:
                    CBi = set()
                    for key, lst in Ci.items():
                        Bi = res1[res1[f'build_existing_model.{key}'].isin(lst)]['building_id']
                        CBi = CBi.union(set(list(Bi)))
                        print(f'    + 3.{k} {key}, {len(Bi)} / {len(res1)}')
                CBi_2all = CBi_2all.union(CBi)
                bldg_list.append(CBi)

                CBi_2len += len(CBi)
                CBi_len += len(CBi)
                print(f'appending total {len(CBi)}, (1.{i})(2.{k})_culm = {CBi_2len}, overall_culm = {CBi_len}\n')

        ## (2) combine partitions, update upgrade name
        P4 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P4.append(p[p['building_id'].isin(Bi)])

        P4 = pd.concat(P4, axis=0).sort_index()

        P4['apply_upgrade.upgrade_part'] = P4['apply_upgrade.upgrade_name'] # new col to show partition #
        P4['apply_upgrade.upgrade_name'] = P4['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P4.columns:
                P4['simulation_output_report.upgrade_cost_usd_orig'] = P4['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P4['simulation_output_report.upgrade_cost_usd'] = P4['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P4['simulation_output_report.upgrade_cost_usd'] -= P4[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P4['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P4.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P4.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P4['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P4.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P4['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, MSHP_cost]

            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P4.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) fix upgrade name
        try:
            P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']] = \
                P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']].apply(
                lambda x: x.str.replace('low-E Window','Low-Gain Window')) # fix upgrade name
        except:
            pass
        
        ## (5) 
        if nullify_total_ng:
            P4.loc[P4['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (6) save
        P4.to_csv(filename, index=False)

    else:              
        P4 = pd.read_csv(filename)
    
    return P4

print('func "combine_df_from_two_level_partitions" loaded')

In [ ]:
pkg_no = 17 # <----
upgrade_list = list(range(5, 5+8)) # <----
MSHP_option = None 
fix_upgrade_name = False
    
## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)



In [ ]:
pkg_no = 18 # <----
upgrade_list = list(range(13, 13+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


## 2B. PACKAGE summary ###

In [9]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

funcs loaded


In [10]:
### set utility rates ###
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''

### utility rates ###########################################
# ref (EIA): 
# avg ComEd res elec rate 2019: $ 0.1330 /kWh
# weighted avg IL gas rate 2019: $ 0.77183 /therm

## electricity ##
# annual fixed rates = monthly x 12
EH_fixed = 15.70 * 12 # annual
NEH_fixed = 14.28 *12 # annual
# avg of summer rates (J,J,A,S) and non-summer rates
EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# marginal carbon emission factor:
elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

## gas ##
# annual fixed rates = monthly x 12
GH_fixed = 45.32 * 12
NGH_fixed = 21.51 * 12
# variable rates
GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# marginal carbon emission factor:
gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

### upgrades that will cause utility rate change: ###########################################
HVAC_upgrades_rate_change = {
    17: [4,6,7], # ASHP
    18: [4,6,7,8], # MSHP,
}

# assign rates accordingly
Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
                      'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
                    'Natural Gas Fuel Boiler']

res['gas_rate'] = NGH_rate
res['gas_fixed'] = NGH_fixed
res['gas_CO2_rate'] = gas_CO2_rate
res['elec_rate'] = NEH_rate
res['elec_fixed'] = NEH_fixed
res['elec_CO2_rate'] = elec_CO2_rate

res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_rate'] = GH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_fixed'] = GH_fixed
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_rate'] = EH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_fixed'] = EH_fixed

# assign 0 gas rates to building with no gas use 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
    res.loc[res['completed_status']!='Success', col] = np.nan

print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


Natural gas rate multiplier: 1


In [11]:
#### count upgrades
N_upgrades = 4 # <---
export_all_upgrades_to_csv = False # <------

#################################################################################################

MF_dict = {
    '2-4 flats': ['Multi-Family with 2 - 4 Units'],
    'all': ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units'],
}

print(f'>>> {iter_path} has {N_upgrades:,} packages')
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
if export_all_upgrades_to_csv:
    print(f'    Exporting upgrades results to {combined_res_csv_path}\n')        
    res.to_csv(os.path.join(combined_res_csv_path, f'results_baseline{fn_ext}.csv'), index=False)
    
## initialize summary tables
summary_upgrades = []

## initialize dB table
all_proto_upgrades = []

package_list = [11, 13, 17, 18] # <-----
for mf_type, MF in MF_dict.items():
    print(f'\n### Summary for {mf_type} ###################################################')
    
    #### *** subset to MF ***
    res_mf = res[res['build_existing_model.geometry_building_type_recs'].isin(MF)].reset_index(drop=True)
    
    for n in package_list:
        p = pd.read_csv(os.path.join(combined_res_csv_path,
                                         f'package{n:02d}.csv'))
        print(f'\nPackage {n}')
        p['build_existing_model.sample_weight'] = 2173432/40000
        
        ### *** subset to MF ***
        p = p[res['build_existing_model.geometry_building_type_recs'].isin(MF)].reset_index(drop=True)

        ### get sim output at unit level
        p = get_per_unit_sim_output(p, res_mf)

        ### assign utility rates
        p = assign_utility_rates_to_upgrade(n, p, res_mf, HVAC_upgrades_rate_change, for_packages=True)

        ### collapse upgrade cost and lifetime cols
        p = combine_upgrade_cost_and_lifetime(p)
        
        ### add totals metrics:
        p = get_annual_totals(p)

        ### check if upgrade has 0 successful sims
        if len(p[p['completed_status']=='Success']) == 0:
            print(f' * upgrade={n} has 0 successful simulations')

        ### calculate metrics
        p = add_sqft_eui(p, res_mf)
        EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']
        EUIo = ['gas_eui','elec_eui','site_eui']
        for vari, varo in zip(EUIi, EUIo):
            p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)

        # annual energy saving:
        p = get_annual_gas_elec_site_energy_saving(p, res_mf)

        # annual energy cost saving:
        p = get_annual_energy_cost_saving(p, res_mf)
        
        # annual kBtu saved per upgrade cost:
        p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
                                p['upgrade_cost'], axis=0)*1000 

        # simple payback
        p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

        # annual metric ton carbon emission savings:
        p = get_annual_metric_ton_co2e_saving(p, res_mf)

        ### check for neg/large paybacks, inf eui, neg carbon savings 
        print_metrics_report(p, has_comparative_payback=False)

        ### subset to only those that have been applied with the upgrades successfully:
        upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
        p = p[p['apply_upgrade.applicable']==True].reset_index(drop=True)
        p['package_no'] = n

        ### export upgrade results to csv 
        if export_all_upgrades_to_csv:
            p.to_csv(os.path.join(combined_res_csv_path, f'results_package{n:02d}{fn_ext}.csv'), index=False)
            
        ### add to dB
        if mf_type == 'all':
            all_proto_upgrades.append(p)

        for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
            p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc

        ### add to summary table
        summ = pd.DataFrame()
        summ.loc[0,'upgrade_no'] = n
        summ.loc[0,'upgrade_name'] = upgrade_name
        summ.loc[0,'mf_type'] = mf_type # <------ *** MF ***
        summ.loc[0,'n_applied'] = len(p)
        summ.loc[0,'n_success'] = len(p[p['completed_status']=='Success'])
        summ.loc[0,'n_fail'] = len(p[p['completed_status']=='Fail'])
        summ['pct_success'] = round(summ['n_success']/summ['n_applied']*100,3)

        p = p[p['completed_status']=='Success'].reset_index(drop=True)
        summ['mean_ann_therm_gas_saving'] = round(p['ann_therm_gas_saving'].mean(), 3)
        summ['mean_ann_kwh_elec_saving'] = round(p['ann_kwh_elec_saving'].mean(), 3)
        summ['mean_ann_mbtu_site_energy_saving'] = round(p['ann_mbtu_site_energy_saving'].mean(), 3)
        summ['mean_pct_delta_gas_eui'] = round(p['pct_delta_gas_eui'].mean(), 3)
        summ['mean_pct_delta_elec_eui'] = round(p['pct_delta_elec_eui'].mean(), 3)
        summ['mean_pct_delta_site_eui'] = round(p['pct_delta_site_eui'].mean(), 3)
        summ['mean_upgrade_cost'] = round(p['upgrade_cost'].mean(),2)
        summ['mean_ann_kbtu_saved_per_dollar'] = round(p['ann_kbtu_saved_per_dollar'].mean(),3) # annual kBtu saved per upgrade cost
        summ['mean_ann_energy_cost_saving'] = round(p['ann_energy_cost_saving'].mean(),2)
        summ['mean_ann_metric_ton_co2e_saving'] = round(p['ann_metric_ton_co2e_saving'].mean(),3)

        summ['median_simple_payback'] = round(p['simple_payback'].median(),3)

        p['simple_payback'] = p['simple_payback'].replace([np.inf, -np.inf], np.nan) # for mean calc
        summ['pct_pos_simple_payback_actual'] = round(len(p[p['simple_payback']>=0])/len(p)*100, 3)
        summ['mean_pos_simple_payback_actual'] = round(p.loc[p['simple_payback']>=0, 'simple_payback'].mean(),3)

        # filter: min 1 cent energy cost savings **
        min_energy_cost_saving = 0.1 # <----- **
        summ['pct_pos_simple_payback_filtered'] = round(len(
            p[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving)]
        )/len(p)*100, 3)
        summ['mean_pos_simple_payback_filtered'] = round(
            p.loc[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving), 
               'simple_payback'].mean(),3)

        summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


>>> cookcnty_packages_03_mf has 4 packages
Natural gas rate multiplier: 1

### Summary for 2-4 flats ###################################################

Package 11
min         8.283089
median     25.920916
max       652.314831
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=11 has 49 simple_payback>100 (including 0 INF)
       *EUI - inf* upgrade=11 has 748 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)

Package 13
min        17.346036
median     35.003526
max       304.649499
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=13 has 4 simple_payback>100 (including 0 INF)
       *EUI - inf* upgrade=13 has 123 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)


/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,130) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 17
min      -15490.798400
median       26.520224
max       41309.734738
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=17 has 341 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=17 has 409 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=17 has 3629 carbon saving<0 

Package 18
min      -25360.132682
median       20.781152
max        4472.710599
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=18 has 66 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=18 has 32 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=18 has 430 carbon saving<0 

### Summary for all ###################################################

Package 11
min          8.283089
median      29.095628
max       1462.509802
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=11 has 169 simple_payback>100 (including 0 INF)

Packa

/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,130) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 17
min      -15490.798400
median       23.196199
max       81064.137850
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=17 has 554 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=17 has 596 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=17 has 7455 carbon saving<0 

Package 18
min      -25360.132682
median       21.414887
max        4472.710599
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=18 has 87 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=18 has 38 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=18 has 698 carbon saving<0 


,upgrade_no,upgrade_name,mf_type,n_applied,n_success,n_fail,pct_success,mean_ann_therm_gas_saving,mean_ann_kwh_elec_saving,mean_ann_mbtu_site_energy_saving,...,mean_pct_delta_site_eui,mean_upgrade_cost,mean_ann_kbtu_saved_per_dollar,mean_ann_energy_cost_saving,mean_ann_metric_ton_co2e_saving,median_simple_payback,pct_pos_simple_payback_actual,mean_pos_simple_payback_actual,pct_pos_simple_payback_filtered,mean_pos_simple_payback_filtered
0,11.0,Comprehensive - Forced Air + WH + Shell,2-4 flats,7712.0,4694.0,0.0,60.866,293.649,693.816,31.725,...,7.357,6539.67,4.631,265.53,2.142,25.921,100.000,31.008,100.000,31.008
1,13.0,Comprehensive - Hydronic + WH + Shell,2-4 flats,7712.0,750.0,0.0,9.725,417.832,518.869,43.544,...,5.231,10917.67,3.867,323.07,2.657,35.004,100.000,38.453,100.000,38.453
2,17.0,Electrification - Gas to ASHP,2-4 flats,7712.0,4694.0,0.0,60.866,934.197,-7257.595,68.268,...,-39.433,11974.79,5.581,351.65,-1.086,26.520,91.287,81.396,91.287,81.396
3,18.0,Electrification - Gas to MSHP,2-4 flats,7712.0,2330.0,0.0,30.213,1104.330,-5733.135,90.485,...,-47.662,12639.96,7.038,574.81,1.092,20.781,98.627,36.398,98.627,36.398
4,11.0,Comprehensive - Forced Air + WH + Shell,all,20759.0,10835.0,0.0,52.194,238.219,623.743,25.945,...,-26.555,6052.51,4.107,223.18,1.788,29.096,100.000,34.537,100.000,34.537
5,13.0,Comprehensive - Hydronic + WH + Shell,all,20759.0,1262.0,0.0,6.079,360.597,509.117,37.788,...,-33.534,10528.84,3.459,286.55,2.344,38.790,100.000,43.039,100.000,43.039
6,17.0,Electrification - Gas to ASHP,all,20757.0,10833.0,0.0,52.190,760.069,-5664.725,56.359,...,-57.747,11376.56,4.822,407.06,-0.683,23.196,94.498,67.817,94.498,67.817
7,18.0,Electrification - Gas to MSHP,all,20759.0,4605.0,0.0,22.183,938.164,-4710.868,77.434,...,-68.014,11828.85,6.354,548.78,1.061,21.415,99.175,31.860,99.175,31.860


In [12]:
### export UPGRADE summary
summary_upgrades.to_csv(os.path.join(result_path,'processed results',f'upgrades_summary{fn_ext}.csv'), index=False)
print(f'UPGRADE summary table saved to:\n  {os.path.join(result_path, "processed results")}')

UPGRADE summary table saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_mf/processed results


In [ ]:
### optional - export specific upgrade parquet as csv
save_to_csv = False # <-----
upgrade_list = range(1, 2+1) # [8] # <----- list of upgrades to convert to csv

if save_to_csv:
    for n in upgrade_list:
        p = pd.read_parquet(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.parquet'))
        p.to_csv(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.csv'), index=False)
        print(f'upgrade {n:02d} parquet file exported as csv')


### *Combine results by building prototypes from Elevate Energy into database

In [13]:
# (1) combine baseline results by non-split-level prototypes (N=12)
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
### add totals metrics:
res = get_annual_totals(res)

res_proto_upgrades = res.copy()

res_proto_upgrades['package_no'] = 0
res_proto_upgrades['apply_upgrade.upgrade_name'] = 'Baseline'

# wt_2_4_flats = 26.4/(26.4+16+8.9)*100
# res_proto_upgrades['build_existing_model.sample_weight'] = np.where(
#     res_proto_upgrades['build_existing_model.geometry_building_type_recs']=='Multi-Family with 2 - 4 Units',
#     wt_2_4_flats,
#     100-wt_2_4_flats
# )

display(res_proto_upgrades)

Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,elec_rate,elec_fixed,elec_CO2_rate,ann_gas_cost,ann_elec_cost,ann_energy_cost,ann_metric_ton_co2e_gas,ann_metric_ton_co2e_elec,ann_metric_ton_co2e,package_no
0,1,54,2021-05-23 13:56:59,2021-05-23 13:58:28,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1499.258711,1542.910428,3042.169139,8.247724,9.071121,17.318844,0
1,2,34,2021-05-23 19:33:59,2021-05-23 19:49:34,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,861.402621,884.863926,1746.266547,2.741383,4.718952,7.460335,0
2,3,5,2021-05-23 17:18:14,2021-05-23 17:21:42,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1312.437599,1594.280303,2906.717902,6.634977,9.410869,16.045846,0
3,4,25,2021-05-23 21:02:19,2021-05-23 21:03:43,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,949.411747,1010.564037,1959.975784,3.501129,5.550303,9.051432,0
4,5,35,2021-05-23 13:04:54,2021-05-23 13:26:08,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,0.000000,1741.850923,1741.850923,0.000000,12.089399,12.089399,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,20755,34,2021-05-24 02:51:17,2021-05-24 02:58:56,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,270.200589,2809.890531,3080.091120,0.113271,20.401188,20.514459,0
20755,20756,95,2021-05-23 12:23:58,2021-05-23 13:09:50,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,269.051469,1543.270619,1812.322089,0.102497,10.543990,10.646487,0
20756,20757,44,2021-05-23 09:49:28,2021-05-23 10:04:24,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,841.639928,931.588438,1773.228366,2.570780,5.027977,7.598757,0
20757,20758,50,2021-05-23 12:15:28,2021-05-23 12:26:20,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,0.000000,2284.110874,2284.110874,0.000000,13.973251,13.973251,0


In [14]:
# (2) combine upgrade results
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')

all_proto_upgrades = pd.concat(all_proto_upgrades, axis=0).reset_index(drop=True)
all_proto_upgrades


Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_mbtu_site_energy_saving,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no
0,1,90,2021-05-24 03:29:56,2021-05-24 03:33:52,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,63.952126,366.364747,167.841987,534.206734,8.023744,14.919992,3.162671,1.110068,4.272740,11
1,2,65,2021-05-23 10:31:26,2021-05-23 10:50:01,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,12.0,...,19.514575,97.734252,135.550339,233.284591,2.908804,28.757993,0.843698,0.896499,1.740197,11
2,3,2,2021-05-23 10:57:25,2021-05-23 11:03:07,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,51.159564,271.659906,262.750924,534.410830,6.093012,15.711569,2.345125,1.737774,4.082899,11
3,4,44,2021-05-23 22:43:46,2021-05-23 22:47:37,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,14.484374,84.709128,27.624995,112.334123,2.300776,56.042009,0.731258,0.182705,0.913963,11
4,5,74,2021-05-23 11:17:26,2021-05-23 11:18:16,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83029,20755,63,2021-05-23 22:34:28,2021-05-23 22:35:14,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
83030,20756,56,2021-05-24 02:23:09,2021-05-24 02:24:05,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
83031,20757,34,2021-05-23 18:48:54,2021-05-23 18:49:46,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
83032,20758,69,2021-05-23 11:00:30,2021-05-23 11:01:21,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18


In [15]:
# (3) combine upgrade and baseline

cols = set(res_proto_upgrades.columns).intersection(set(all_proto_upgrades.columns))
all_proto_upgrades = pd.concat([all_proto_upgrades, res_proto_upgrades[cols]], axis=0).reset_index(drop=True)

# (4) save
print('"all_proto_upgrades" df saved')
all_proto_upgrades.to_csv(
    os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv'), index=False)

display(all_proto_upgrades)


"all_proto_upgrades" df saved


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_mbtu_site_energy_saving,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no
0,1,90,2021-05-24 03:29:56,2021-05-24 03:33:52,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.000000,...,63.952126,366.364747,167.841987,534.206734,8.023744,14.919992,3.162671,1.110068,4.272740,11
1,2,65,2021-05-23 10:31:26,2021-05-23 10:50:01,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,12.000000,...,19.514575,97.734252,135.550339,233.284591,2.908804,28.757993,0.843698,0.896499,1.740197,11
2,3,2,2021-05-23 10:57:25,2021-05-23 11:03:07,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.000000,...,51.159564,271.659906,262.750924,534.410830,6.093012,15.711569,2.345125,1.737774,4.082899,11
3,4,44,2021-05-23 22:43:46,2021-05-23 22:47:37,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.000000,...,14.484374,84.709128,27.624995,112.334123,2.300776,56.042009,0.731258,0.182705,0.913963,11
4,5,74,2021-05-23 11:17:26,2021-05-23 11:18:16,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103788,20755,34,2021-05-24 02:51:17,2021-05-24 02:58:56,Success,None,Baseline,,True,20.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
103789,20756,95,2021-05-23 12:23:58,2021-05-23 13:09:50,Success,None,Baseline,,True,6.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
103790,20757,44,2021-05-23 09:49:28,2021-05-23 10:04:24,Success,None,Baseline,,True,4.285714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
103791,20758,50,2021-05-23 12:15:28,2021-05-23 12:26:20,Success,None,Baseline,,True,12.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [16]:
## save metadata for dB
if NG_rate_multiplier==1:
    cols2 = sorted((set(res_proto_upgrades.columns)-set(cols)).union(
                   set(['building_id','completed_status','package_no']))
                  )
    res_proto_meta = res_proto_upgrades[cols2]
    res_proto_meta = res_proto_meta[res_proto_meta['completed_status']=='Success'].reset_index(drop=True)
    
    cols_to_drop = [x for x in res_proto_meta.columns if 
                    x.startswith('simulation_output_report.option') or
                    x == 'simulation_output_report.upgrade_cost_usd']
    
    res_proto_meta = res_proto_meta.drop(cols_to_drop, axis=1)

    res_proto_meta.to_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv'), index=False)

    print('"res_proto_meta" df saved')
    display(res_proto_meta)
    

"res_proto_meta" df saved


,build_existing_model.ahs_region,build_existing_model.applicable,build_existing_model.ashrae_iecc_climate_zone_2004,build_existing_model.bathroom_spot_vent_hour,build_existing_model.bedrooms,build_existing_model.building_america_climate_zone,build_existing_model.ceiling_fan,build_existing_model.census_division,build_existing_model.census_region,build_existing_model.chicago,...,build_existing_model.vintage,build_existing_model.vintage_acs,build_existing_model.water_heater,build_existing_model.window_areas,build_existing_model.windows,building_id,completed_status,package_no,stories,vintage_ee
0,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour14,3,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1940s,1940-59,Gas Standard,F12 B12 L12 R12,2+ Pane,1,Success,0,1: <2 stories,2: 1942-1978
1,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1950s,1940-59,Gas Standard,F30 B30 L30 R30,2+ Pane,2,Success,0,2: 2+ stories,2: 1942-1978
2,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour22,5,Cold,None,East North Central,Midwest,City of Chicago,...,<1940,<1940,Gas Standard,F9 B9 L9 R9,1 Pane,3,Success,0,1: <2 stories,1: pre-1942
3,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1950s,1940-59,Gas Standard,F9 B9 L9 R9,1 Pane,4,Success,0,1: <2 stories,2: 1942-1978
4,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour1,2,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1990s,1980-99,Electric Standard,F15 B15 L15 R15,2+ Pane,5,Success,0,2: 2+ stories,3: post-1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,2,Cold,Standard Efficiency,East North Central,Midwest,Non-City Cook County,...,1970s,1960-79,Electric Standard,F18 B18 L18 R18,2+ Pane,20755,Success,0,2: 2+ stories,2: 1942-1978
20755,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,1,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,1990s,1980-99,Electric Standard,F30 B30 L30 R30,2+ Pane,20756,Success,0,2: 2+ stories,3: post-1978
20756,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,3,Cold,Standard Efficiency,East North Central,Midwest,City of Chicago,...,2000s,2000-09,Gas Standard,F18 B18 L18 R18,2+ Pane,20757,Success,0,2: 2+ stories,3: post-1978
20757,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour22,2,Cold,Standard Efficiency,East North Central,Midwest,Non-City Cook County,...,1980s,1980-99,Electric Standard,F15 B15 L15 R15,2+ Pane,20758,Success,0,2: 2+ stories,3: post-1978


## **Get core bldg aggregates for Elevate

In [17]:
load_db_from_local = True # <-----
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''
    
if load_db_from_local:
    all_proto_upgrades = pd.read_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv')
    )
    
    res_proto_meta = pd.read_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv')
    )
    print('"all_proto_upgrades" and "res_proto_meta" loaded.')

/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,58,105) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


"all_proto_upgrades" and "res_proto_meta" loaded.


### get 2 packages from packages_02 run and combine
* Comprehensive - ASHP
* Comprehensive - MSHP

In [18]:
result_path2 = os.path.join(result_dir, f'cookcnty_packages_02_mf') # <-----

if load_db_from_local:
    all_proto_upgrades1 = pd.read_csv(
        os.path.join(result_path2,'processed results',f'upgrades_dB{fn_ext}.csv')
    )
    
    print('"all_proto_upgrades1" loaded.')

/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,58,139) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


"all_proto_upgrades1" loaded.


In [19]:
upgrades_to_retrieve = ['Comprehensive - ASHP', 'Comprehensive - MSHP'] # <----

all_proto_upgrades1 = all_proto_upgrades1[
    all_proto_upgrades1['apply_upgrade.upgrade_name'].isin(upgrades_to_retrieve)]

# combine with other packages
all_proto_upgrades = pd.concat([
    all_proto_upgrades,
    all_proto_upgrades1
], axis=0)

# check upgrades:
print('Packages combined:')
all_proto_upgrades['apply_upgrade.upgrade_name'].unique()

# check all bldgs are MF


Packages combined:


array(['Comprehensive - Forced Air + WH + Shell',
       'Comprehensive - Hydronic + WH + Shell',
       'Electrification - Gas to ASHP', 'Electrification - Gas to MSHP',
       'Baseline', 'Comprehensive - ASHP', 'Comprehensive - MSHP'],
      dtype=object)

### Select Type 1 or Type 2 below:
* Type 1: data for Anna's city-scale analysis (MF 2-4 Flat Frame/Masonry by # of unit)
* Type 2: data for Honnie's graphics (MF 204 Flat Frame/Masonry Pre-1942)

In [20]:
consolidate_as_type1 = True # <------

res_proto_meta2 = res_proto_meta.copy()

if consolidate_as_type1:
    msg = "Type 1: data for Anna's city-scale analysis (MF 2-4 Flat Frame/Masonry Pre-1942 by no. of unit)"
    ext = "_pre-1942_by_unit_count"
    
#     res_proto_meta2['bldg_segment'] = \
#         'MF '+ res_proto_meta2['build_existing_model.geometry_building_type_recs'].replace(
#         ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units'], ['2-4 Flat', '5 plus']) + ' ' + \
#         res_proto_meta2['build_existing_model.geometry_wall_type'].replace('WoodStud', 'Frame')


#     all_proto_upgrades2 = all_proto_upgrades.loc[
#         all_proto_upgrades['simulation_output_report.applicable']==True
#     ].join(
#         res_proto_meta2.set_index('building_id')[[
#             'bldg_segment', 'build_existing_model.geometry_building_number_units_mf'
#         ]], on = 'building_id')


#     # core bldg types that Elevate is interested in:
#     core_bldgs = ['MF 2-4 Flat Frame', 'MF 2-4 Flat Masonry']
#     all_proto_upgrades2 = all_proto_upgrades2.loc[all_proto_upgrades2['bldg_segment'].isin(core_bldgs)]

else:
    msg = "Type 2: data for Honnie's graphics (MF 2-4 Flat Frame/Masonry Pre-1942)"
    ext = "_pre-1942_combined"
    
res_proto_meta2['bldg_segment'] = \
    'MF '+ res_proto_meta2['build_existing_model.geometry_building_type_recs'].replace(
    ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units'], ['2-4 Flat', '5 plus']) + ' ' + \
    res_proto_meta2['build_existing_model.geometry_wall_type'].replace('WoodStud', 'Frame') + ' ' + \
    res_proto_meta2['vintage_ee'].str[3:].str.title()

all_proto_upgrades2 = all_proto_upgrades.loc[
    all_proto_upgrades['simulation_output_report.applicable']==True
].join(
    res_proto_meta2.set_index('building_id')[[
        'bldg_segment', 'build_existing_model.geometry_building_number_units_mf'
    ]], on = 'building_id')


# core bldg types that Elevate is interested in:
core_bldgs = ['MF 2-4 Flat Frame Pre-1942', 'MF 2-4 Flat Masonry Pre-1942']
all_proto_upgrades2 = all_proto_upgrades2.loc[all_proto_upgrades2['bldg_segment'].isin(core_bldgs)]

print('> Get distribution of MF Frame/Masonry Pre-1942:')
display(
    all_proto_upgrades2.groupby(['apply_upgrade.upgrade_name',
                                 'bldg_segment'])['building_id'].count()
)

# get ann_energy_cost_saving to bldg level (not unit)
all_proto_upgrades2['ann_energy_cost_saving_bldg'] = all_proto_upgrades2['ann_energy_cost_saving'] * \
    all_proto_upgrades2['build_existing_model.geometry_building_number_units_mf'].astype('f')
    
print(f'>> Consolidating for {msg}')
all_proto_upgrades2

> Get distribution of MF Frame/Masonry Pre-1942:


apply_upgrade.upgrade_name               bldg_segment                
Baseline                                 MF 2-4 Flat Frame Pre-1942      1470
                                         MF 2-4 Flat Masonry Pre-1942    2614
Comprehensive - ASHP                     MF 2-4 Flat Frame Pre-1942       807
                                         MF 2-4 Flat Masonry Pre-1942    1410
Comprehensive - Forced Air + WH + Shell  MF 2-4 Flat Frame Pre-1942       807
                                         MF 2-4 Flat Masonry Pre-1942    1410
Comprehensive - Hydronic + WH + Shell    MF 2-4 Flat Frame Pre-1942       181
                                         MF 2-4 Flat Masonry Pre-1942     336
Comprehensive - MSHP                     MF 2-4 Flat Frame Pre-1942       566
                                         MF 2-4 Flat Masonry Pre-1942    1032
Electrification - Gas to ASHP            MF 2-4 Flat Frame Pre-1942       807
                                         MF 2-4 Flat Masonry Pre-1942   

>> Consolidating for Type 1: data for Anna's city-scale analysis (MF 2-4 Flat Frame/Masonry by no. of unit)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no,bldg_segment,build_existing_model.geometry_building_number_units_mf,ann_energy_cost_saving_bldg
2,3,2,2021-05-23 10:57:25,2021-05-23 11:03:07,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,534.410830,6.093012,15.711569,2.345125,1.737774,4.082899,11,MF 2-4 Flat Masonry Pre-1942,4,2137.643318
5,6,67,2021-05-24 00:25:24,2021-05-24 00:29:01,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,406.535923,6.923168,18.164016,2.523586,0.908674,3.432260,11,MF 2-4 Flat Frame Pre-1942,2,813.071847
10,11,59,2021-05-24 01:53:55,2021-05-24 01:58:56,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,227.207322,3.278660,29.887549,1.029848,0.713689,1.743536,11,MF 2-4 Flat Masonry Pre-1942,2,454.414644
22,23,83,2021-05-23 21:33:12,2021-05-23 21:39:21,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,594.108805,8.117954,15.387661,3.714742,1.083283,4.798025,11,MF 2-4 Flat Frame Pre-1942,4,2376.435222
36,37,51,2021-05-23 11:33:20,2021-05-23 11:38:55,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,20.0,...,274.994133,6.200013,22.940199,2.023573,0.268406,2.291979,11,MF 2-4 Flat Masonry Pre-1942,3,824.982399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228286,20699,121,2021-02-14 20:01:41,2021-02-14 20:07:23,Success,True,Comprehensive - MSHP,NaN,True,20.0,...,604.435774,10.190732,15.653330,5.861497,-3.350382,2.511115,16,MF 2-4 Flat Masonry Pre-1942,4,2417.743097
228300,20713,101,2021-02-14 08:08:31,2021-02-14 08:14:42,Success,True,Comprehensive - MSHP,NaN,True,20.0,...,544.726171,8.127105,17.260464,4.709751,-2.951446,1.758305,16,MF 2-4 Flat Masonry Pre-1942,3,1634.178512
228315,20728,185,2021-02-14 08:22:18,2021-02-14 08:27:26,Success,True,Comprehensive - MSHP,NaN,True,20.0,...,149.264760,7.560545,83.476133,5.920627,-4.163776,1.756852,16,MF 2-4 Flat Masonry Pre-1942,2,298.529519
228323,20736,220,2021-02-14 12:06:12,2021-02-14 12:12:31,Success,True,Comprehensive - MSHP,NaN,True,20.0,...,169.885138,7.245758,81.271294,6.349285,-4.706095,1.643190,16,MF 2-4 Flat Frame Pre-1942,4,679.540553


In [21]:
## get package-specific baseline, remove generic baseline

upgrade_list = [x for x in all_proto_upgrades2['apply_upgrade.upgrade_name'].unique() if x != 'Baseline']

# query package-specific baselines
upgrade_spec_baselines = []
for up in upgrade_list:
    applicable_bldgs = all_proto_upgrades2.loc[
        all_proto_upgrades2['apply_upgrade.upgrade_name']==up,
        'building_id'
    ]
    
    baseline_up = all_proto_upgrades2.loc[(all_proto_upgrades2['apply_upgrade.upgrade_name']=='Baseline') & 
                                          (all_proto_upgrades2['building_id'].isin(applicable_bldgs))
                                         ]
    baseline_up['apply_upgrade.upgrade_name'] += ': '+up
    
    upgrade_spec_baselines.append(baseline_up)
    
# remove generic baseline and combine
upgrade_spec_baselines.append(
    all_proto_upgrades2.loc[all_proto_upgrades2['apply_upgrade.upgrade_name']!='Baseline']
)

all_proto_upgrades2 = pd.concat(upgrade_spec_baselines, axis=0)
del upgrade_spec_baselines

# check that package-specific baselines exist
all_proto_upgrades2['apply_upgrade.upgrade_name'].value_counts().sort_index()
                                          

<ipython-input-21-f37e80ce4334>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_up['apply_upgrade.upgrade_name'] += ': '+up


Baseline: Comprehensive - ASHP                       2217
Baseline: Comprehensive - Forced Air + WH + Shell    2217
Baseline: Comprehensive - Hydronic + WH + Shell       517
Baseline: Comprehensive - MSHP                       1598
Baseline: Electrification - Gas to ASHP              2217
Baseline: Electrification - Gas to MSHP              1598
Comprehensive - ASHP                                 2217
Comprehensive - Forced Air + WH + Shell              2217
Comprehensive - Hydronic + WH + Shell                 517
Comprehensive - MSHP                                 1598
Electrification - Gas to ASHP                        2217
Electrification - Gas to MSHP                        1598
Name: apply_upgrade.upgrade_name, dtype: int64

In [22]:
metrics = {
    'ann_gas_cost_saving': 'Ann Gas Cost Saving',
    'ann_elec_cost_saving': 'Ann Elec Cost Saving',
    'ann_energy_cost_saving': 'Annual Utility Bill Savings',
    
    'ann_therm_gas_saving': 'Ann Therm Gas Saving', 
    'ann_kwh_elec_saving': 'Ann Kwh Elec Saving',
    'ann_mbtu_site_energy_saving': 'Ann Mbtu Site Energy Saving',
    'ann_metric_ton_co2e_saving': 'Ann Metric Ton Co2E Saving',
    'pct_delta_site_eui': 'Pct Delta Site Eui',
    
    'simulation_output_report.total_site_natural_gas_therm': 'Ann Gas Use (Therms)',
    'simulation_output_report.total_site_electricity_kwh': 'Ann Elec Use (kWh)',
    'simulation_output_report.total_site_energy_mbtu': 'Ann Site Energy (mmBtu)',
    
    'ann_gas_cost': 'Ann Gas Cost',
    'ann_elec_cost': 'Ann Elec Cost',
    'ann_energy_cost': 'Ann Energy Cost',
    'ann_metric_ton_co2e': 'Ann Metric Ton Co2E',
    'site_eui_kbtupersqft': 'Site Eui',
    'upgrade_cost': 'Upgrade Cost',
} # per unit

# define groupby
if consolidate_as_type1:
    groupby_cols = ['apply_upgrade.upgrade_name',
                    'bldg_segment',
                    'build_existing_model.geometry_building_number_units_mf']
else:
    groupby_cols = ['apply_upgrade.upgrade_name',
                    'bldg_segment']

# get count
count = all_proto_upgrades2.groupby(groupby_cols)[
    'building_id'].count().rename('count').to_frame()

# for each metric, get [P25, avg, median, P75]
all_proto_summ = all_proto_upgrades2.groupby(groupby_cols)[
    list(metrics.keys())].agg([
    'min',
    lambda x: x.quantile(0.25),
    'mean',
    'median',
    lambda x: x.quantile(0.75),
    'max',
])

# rename cols
all_proto_summ = all_proto_summ.rename(columns = metrics, level=0)

all_proto_summ = all_proto_summ.rename(columns = {
    'min': 'Min.',
    '<lambda_0>': 'Percentile (25) of',
    'mean': 'Avg.',
    'median': 'Median',
    '<lambda_1>': 'Percentile (75) of',
    'max': 'Max.',
}, level=1)

all_proto_summ.columns = [" ".join(x) for x in all_proto_summ.columns.swaplevel().ravel()]

# combine and transpose
all_proto_summ = pd.concat([count, all_proto_summ], axis=1).transpose()
del count

# export
all_proto_summ.to_csv(
    os.path.join(result_path,'processed results',f'summary_elevate_core_bldgs_mf{ext}.csv'), 
    index=True)

print(f'>> Summary for {msg}')
all_proto_summ

>> Summary for Type 1: data for Anna's city-scale analysis (MF 2-4 Flat Frame/Masonry by no. of unit)


apply_upgrade.upgrade_name                             Baseline: Comprehensive - ASHP  \
bldg_segment                                               MF 2-4 Flat Frame Pre-1942   
build_existing_model.geometry_building_number_units_mf                              2   
count                                                                           430.0   
Min. Ann Gas Cost Saving                                                          NaN   
Percentile (25) of Ann Gas Cost Saving                                            NaN   
Avg. Ann Gas Cost Saving                                                          NaN   
Median Ann Gas Cost Saving                                                        NaN   
...                                                                               ...   
Percentile (25) of Upgrade Cost                                                   NaN   
Avg. Upgrade Cost                                                                 NaN   
Median Upgrade Cost                                                               NaN   
Percentile (75) of Upgrade Cost                                                   NaN   
Max. Upgrade Cost                                                                 NaN   

apply_upgrade.upgrade_name                                            \
bldg_segment                                                           
build_existing_model.geometry_building_number_units_mf      3      4   
count                                                   181.0  196.0   
Min. Ann Gas Cost Saving                                  NaN    NaN   
Percentile (25) of Ann Gas Cost Saving                    NaN    NaN   
Avg. Ann Gas Cost Saving                                  NaN    NaN   
Median Ann Gas Cost Saving                                NaN    NaN   
...                                                       ...    ...   
Percentile (25) of Upgrade Cost                           NaN    NaN   
Avg. Upgrade Cost                                         NaN    NaN   
Median Upgrade Cost                                       NaN    NaN   
Percentile (75) of Upgrade Cost                           NaN    NaN   
Max. Upgrade Cost                                         NaN    NaN   

apply_upgrade.upgrade_name                                                           \
bldg_segment                                           MF 2-4 Flat Masonry Pre-1942   
build_existing_model.geometry_building_number_units_mf                            2   
count                                                                         765.0   
Min. Ann Gas Cost Saving                                                        NaN   
Percentile (25) of Ann Gas Cost Saving                                          NaN   
Avg. Ann Gas Cost Saving                                                        NaN   
Median Ann Gas Cost Saving                                                      NaN   
...                                                                             ...   
Percentile (25) of Upgrade Cost                                                 NaN   
Avg. Upgrade Cost                                                               NaN   
Median Upgrade Cost                                                             NaN   
Percentile (75) of Upgrade Cost                                                 NaN   
Max. Upgrade Cost                                                               NaN   

apply_upgrade.upgrade_name                                            \
bldg_segment                                                           
build_existing_model.geometry_building_number_units_mf      3      4   
count                                                   325.0  320.0   
Min. Ann Gas Cost Saving                                  NaN    NaN   
Percentile (25) of Ann Gas Cost Saving                    NaN    NaN   
Avg. Ann Gas Cost Saving                                  NaN    NaN   
Median Ann Gas Cost S

### 2.3. Visualize UPGRADE prototype summary 
#### 2.3.1. plot mean metrics values
Run cell below to load df for plots

In [ ]:
load_df_for_plot_below_from_file = False # <-----
    
if load_df_for_plot_below_from_file:
    
    NG_rate_multiplier = 1 # 1 or 3 <----- This controls what to plot in '2.3.5.'
 
    if NG_rate_multiplier > 1:
        fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
    else:
        fn_ext = ''
    
    filename = os.path.join(result_path,'processed results', f'upgrades_summary{fn_ext}.csv')
    if not os.path.exists(filename):
        print(f'"summary_upgrades{fn_ext}" df does not exist, check if Multi-Family buildings have been modeled.')
    else:
        summary_upgrades = pd.read_csv(filename)
    
        print(f'"summary_upgrades{fn_ext}" df loaded from file.')
        N_upgrades = summary_upgrades['upgrade_no'].nunique()
        print(f'   total number of upgrades: {N_upgrades}')

        display(summary_upgrades)
    

In [ ]:
### plot MF-all & MF-2-4 flats (Cook County Avg) ###

for mf_type in summary_upgrades['mf_type'].unique():
    summ_ups = summary_upgrades[
        (~summary_upgrades['upgrade_no'].isnull()) & (summary_upgrades['mf_type']==mf_type)
         ].reset_index(drop=True)

    N_upgrades=15
    upgrade_sets = {
        'Packages': range(6, N_upgrades+6),
    }
    upgrade_set_colors = {
        'Packages': 'tab:blue',
    }
    # create subfolder for plots:

    plot_path5 = os.path.join(plot_path, f'mean_values_mf_{mf_type}_cook_cnty_avg')
    if not os.path.exists(plot_path5):
        os.mkdir(plot_path5)
    print(f'plot dir: {plot_path5}\n')

    if NG_rate_multiplier == 1: 
        metrics_to_plot = list(x for x in summary_upgrades.columns if 
                               x.startswith('mean') or x.startswith('median'))
    else:
        metrics_to_plot = ['mean_ann_energy_cost_saving', 
                           'median_simple_payback',
                           'mean_pos_simple_payback_filtered']

    for metric in metrics_to_plot:
        title_ext = '' if fn_ext == '' else f' ({fn_ext.replace("_"," ").lstrip()})'
        title = f'Multi-Family {mf_type} per Unit: {metric}{title_ext}'
        print(f'Plotting {title}...')

        for n, up in enumerate(upgrade_sets.keys(),1):
            print(f'    - Group{n}')

            groupi = summ_ups.loc[:, ['upgrade_name', metric]
                                 ].set_index(['upgrade_name']).replace([np.inf, -np.inf], np.nan)

            if metric == 'median_comparative_payback':
                groupi = groupi.replace(0,np.nan).sort_values(by = metric) # make 0 nan
            else:
                groupi = groupi.sort_values(by = metric)
            groupi.index.name = up

            ht = len(groupi)
            fig, ax = plt.subplots(figsize=(6, 0.3*ht))
            groupi.plot.barh(legend=False, title=title, ax=ax)

            # For each bar: Place a label ############################
            rects = ax.patches
            for rect in rects:
                # Get X and Y placement of label from rect.
                x_value = rect.get_width()
                y_value = rect.get_y() + rect.get_height() / 2

                # Number of points between bar and label. Change to your liking.
                space = 5
                # Vertical alignment for positive values
                ha = 'left'

                # If value of bar is negative: Place label left of bar
                if x_value < 0:
                    # Invert space to place label to the left
                    space *= -1
                    # Horizontally align label at right
                    ha = 'right'

                # Use X value as label and format number with one decimal place
                label = "{:.1f}".format(x_value)

                # Create annotation
                plt.annotate(
                    label,                      # Use `label` as label
                    (x_value, y_value),         # Place label at end of the bar
                    xytext=(space, 0),          # Horizontally shift label by `space`
                    textcoords="offset points", # Interpret `xytext` as offset in points
                    va='center',                # Vertically center label
                    ha=ha,                      # Horizontally align label differently for
                    fontsize=8)                 # positive and negative values. 

            ax.margins(x=0.2)    
            ax.axvline(x=0, linestyle='-', color='gray')
            if (metric[-3:]=='eui') & (ax.get_xlim()[0] < -50):
                ax.axvline(x=-50, linestyle='--', color='darkred')

            filename = f'MF-{mf_type}_upgrades_{iteration}_{metric[5:]}_group{n}{fn_ext}.pdf'
            fig.savefig(os.path.join(plot_path5, filename),
                        bbox_inches='tight')
        